In [1]:
import shutil
import pathlib
import subprocess
import shlex

from sklearn.decomposition import PCA
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm.notebook as tqdm
import patchworklib as pw 

<Figure size 100x100 with 0 Axes>

In [2]:
shutil.rmtree("data/pheno", ignore_errors=True)
pheno_root = pathlib.Path("data/pheno")
pheno_root.mkdir(exist_ok=True)

# Select ICD-10 features

In [3]:
min_sample_size = 1000

columns_to_keep = (
    pl.scan_csv("../../data/pheno_jan2024.tsv", separator="\t")
    .select(pl.col("^b_.+$").sub(2).sum())
    .unpivot()
    .filter(pl.col("value").ge(min_sample_size))
    .collect()
    ["variable"]
    .to_list()
)
print(f"Keeping {len(columns_to_keep)} ICD-10 codes")

(
    pl.scan_csv("../../data/pheno_jan2024.tsv", separator="\t")
    .select("#FID", "IID", pl.col(columns_to_keep).name.map(lambda x: x.replace("b_", "")))
    .sink_csv(pheno_root / "icd.tsv", separator="\t", null_value="NA")
)

Keeping 608 ICD-10 codes


# Compute PCA phenotypes

In [4]:
pheno_df = pl.read_csv(pheno_root / "icd.tsv", separator="\t", null_values=["NA"])

X = (
    pheno_df
    .select(pl.col("^[A-Z][0-9]{2}$"))
    .to_pandas()
    .astype(float)
)
fid_iid = pheno_df.select("#FID", "IID")
column_names = X.columns.tolist()
n_codes = len(column_names)

fractions = [10, 25, 50, 75, 90]

for fraction in tqdm.tqdm(fractions):
    n_to_keep = int(n_codes * fraction / 100)
    print(f"Fraction: {fraction}, Keeping: {n_to_keep}")
    pca = PCA(n_components=n_to_keep)
    pcs = pca.fit_transform(X)
    Xhat = pca.inverse_transform(pcs)
    Xhat_df = pl.concat([
        fid_iid,
        pl.DataFrame(Xhat, schema=column_names)
    ], how="horizontal")
    Xhat_df.write_csv(pheno_root / f"icd_pca_{fraction}.tsv", separator="\t", null_value="NA")

  0%|          | 0/5 [00:00<?, ?it/s]

Fraction: 10, Keeping: 60
Fraction: 25, Keeping: 152
Fraction: 50, Keeping: 304
Fraction: 75, Keeping: 456
Fraction: 90, Keeping: 547


# GWAS

In [5]:
shutil.rmtree("data/gwas", ignore_errors=True)
gwas_root = pathlib.Path("data/gwas")
gwas_root.mkdir(exist_ok=True)

In [6]:
for path in pheno_root.glob("*.tsv"):
    output_root = gwas_root / path.stem
    output_root.mkdir(exist_ok=True)

    # For fast testing, add --thin-indiv-count 10000
    command = f"""
    plink2 \
      --pfile ../../data/geno/ukb_wb_subsampled \
      --pheno {path.as_posix()} \
      --no-input-missing-phenotype \
      --glm allow-no-covars hide-covar zs \
      --threads 110 \
      --out {output_root.joinpath('result').as_posix()}
    """
    subprocess.run(shlex.split(command))

PLINK v2.0.0-a.6.0LM AVX2 Intel (11 Nov 2024)      cog-genomics.org/plink/2.0/
(C) 2005-2024 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to data/gwas/icd/result.log.
Options in effect:
  --glm allow-no-covars hide-covar zs
  --no-input-missing-phenotype
  --out data/gwas/icd/result
  --pfile ../../data/geno/ukb_wb_subsampled
  --pheno data/pheno/icd.tsv
  --threads 110

Start time: Sat Dec  7 17:21:27 2024
1031943 MiB RAM detected, ~949202 available; reserving 515971 MiB for main
workspace.
Using up to 110 threads (change this with --threads).
429954 samples (232741 females, 197213 males; 429954 founders) loaded from
../../data/geno/ukb_wb_subsampled.psam.
10000 variants loaded from ../../data/geno/ukb_wb_subsampled.pvar.
608 quantitative phenotypes loaded.
Calculating allele frequencies... done.
--glm linear regression on quantitative phenotypes #1-240: done.
--glm linear regression on quantitative phenotypes #241-480: done.
--glm linear regression on quan

# Gather GWAS

In [7]:
(
    pl.scan_csv("data/gwas/*/result.*.glm.linear.zst", separator="\t", glob=True, include_file_paths="path")
    .select(
        (
            pl.col("path")
            .str.strip_prefix("data/gwas/")
            .str.strip_suffix(".glm.linear.zst")
        ),
        pl.col("ID").alias("variant_id"),
        pl.col("T_STAT").pow(2).alias("chisq"),
    )
    .select(
        pl.col("path").str.extract("icd_pca_([0-9]{2})").fill_null("100").alias("fraction"),
        pl.col("path").str.extract("([A-Z][0-9]{2})$").alias("phenotype"),
        "variant_id",
        "chisq",
    )
    .sink_parquet(gwas_root / "full_gwas.parquet")
)

# Compute GWAS fit

In [8]:
results_df = pl.scan_parquet(gwas_root / "full_gwas.parquet")
true_df = (
    results_df
    .filter(pl.col("fraction").eq("100"))
    .drop(["fraction"])
    .rename({"chisq": "chisq_true"})
)
est_df = (
    results_df
    .filter(pl.col("fraction").ne("100"))
    .rename({"chisq": "chisq_est"})
)
(
    true_df
    .join(est_df, on=["phenotype", "variant_id"])
    .sink_parquet(gwas_root / "comparison.parquet")
)

In [9]:
(
    pl.scan_parquet(gwas_root / "comparison.parquet")
    .group_by(["phenotype", "fraction"])
    .agg(pl.corr("chisq_true", "chisq_est").alias("r"))
    .with_columns(pl.col("r").pow(2).alias("rsq"))
    .collect()
    .write_parquet("data/gwas_summary.parquet")
)

# Compute phenotype fit

In [10]:
full_pheno_df = pl.scan_csv(pheno_root / "icd.tsv", separator="\t", null_values=["NA"])
icd_codes = full_pheno_df.drop(["#FID", "IID"]).collect_schema().names()

approx_pheno_paths = list(pheno_root.glob("icd_*.tsv"))

pheno_fit_results = list()
for approx_pheno_path in approx_pheno_paths:
    fraction = approx_pheno_path.stem.split("_")[-1]
    
    approx_pheno_df = (
        full_pheno_df
        .join(
            pl.scan_csv(approx_pheno_path, separator="\t", null_values=["NA"]),
            on=["#FID", "IID"], suffix="_est"
        )
    )

    this_summary_df = (
        approx_pheno_df
        .select([
            pl.corr(icd_code, f"{icd_code}_est").alias(icd_code)
            for icd_code in icd_codes
        ])
        .unpivot(variable_name="phenotype", value_name="r")
        .select(
            pl.lit(fraction).alias("fraction"),
            "phenotype",
            "r",
            pl.col("r").pow(2).alias("rsq"),
        )
    )
    pheno_fit_results.append(this_summary_df)

(
    pl.concat(pheno_fit_results)
    .collect()
    .write_parquet("data/pheno_summary.parquet")
)